# Donnée de stock

Dans les données de stock, il y a un stock inital a date du 5 mai 2017 et un ensemble de stock initial délivré à différent intervals.



## Documentation

- [Doc INSEE](https://scm.saas.cagip.group.gca/PERNETTH/database/blob/master/Documentation/IMR/Doc_Tech_IMR_Mai_2019_v1.5.1.pdf)
- [etalab](https://github.com/etalab/rncs_worker_api_entreprise)
    - [Module data cleaning](https://github.com/etalab/rncs_worker_api_entreprise/blob/master/clean_csv.bash)
- Path local: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Stock\Stock_initial`
- Path Reseau: `G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\stock\2017`

### Variables communes

![image](https://drive.google.com/uc?export=view&id=1laCGFEtxM4bbckNZQEkw1icBjF4pd20-)

### Stock

Le contenu de chaque fichier est disponible [ici](https://scm.saas.cagip.group.gca/PERNETTH/database/tree/master/INPI/TC_1)

Les données brutes sont répertoriées dans des fichiers zip composées de 7 fichiers:

- Actes
- Comptes Annuels
- ETS
- OBS
- PM
- PP
- REP

### Donnees de flux 

- Path local: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux`
- Path Reseau: `G:\Projets\PP2019\2019_PROGRAMME BATICA\DATA\INPI\sources\IMR_Donnees_Saisies\tc\Flux`

Les PP en local sont disponibles ici: `C:\Users\PERNETTH\Documents\Projects\GitLab\CreationDataBase\INPI\TC_1\Flux\PP`

name csv : `0101_1_20170512_112544_4_PP_EVT`
    
    - code greffe: 0101
    - numéro transmission: 1
    - Date: 2017-05-12
    - heures: 11h25m44s
    - Type: PP
    - Evenement: Oui

**FICHIERS TRANSMIS EN CAS DE MISE A JOUR D’UN DOSSIER (EVENEMENT)**

En cas de mise à jour d’un dossier suite à un événement (modification, radiation), les fichiers transmis ont une structure identique aux fichiers créés à l’immatriculation avec la présence de 2 champs spécifiques : la date de l’événement (Date_Greffe) et le libellé de l’événement (Libelle_Evt).

Dans ces cas, 6 types de fichiers supplémentaires, numérotés, sont transmis correspondant à :
* Evénements modifiant ou complétant les dossiers d’immatriculation des personnes morales (2) ou physiques (4)
* Evénements modifiant ou complétant les informations relatives aux représentants (6) ou aux établissements (9)
* Evénements supprimant des représentants (7 – Représentant partant) ou des établissements (10 – Etablissement supprimé)

![](https://drive.google.com/uc?export=view&id=1FVEGNqogl1NxB84BtdF4TQztCXjmtpyo)

Attention, il peut arriver que le même dossier fasse l’objet de plusieurs événements (création et modification) dans la même transmission. Il est impératif d’intégrer les événements dans l’ordre d’apparition.

Les csv sont classés de la manière suivante:

- Module (ETS, PP, PM, etc)
    - NEW: New event
    - EVT: Update event

In [ ]:
import boto3

import dask.dataframe as dd
import pandas as pd
import Match_inpi_insee.inpi as inpi
%load_ext autoreload
%autoreload 2

## PP

In [ ]:
dtype = {
    "Code Greffe": "int",
    "Nom_Greffe": "str",
    "Numero_Gestion": "str",
    "Siren": "int",
    "Type_Inscription": "str",
    "Date_Immatriculation": "str",
    "Date_1re_Immatriculation": "str",
    "Date_Radiation": "str",
    "Date_Transfert": "str",
    "Sans_Activité": "str",
    "Date_Debut_Activité": "str",
    "Date_Début_1re_Activité": "str",
    "Date_Cessation_Activité": "str",
    "Nom_Patronymique": "str",
    "Nom_Usage": "str",
    "Pseudonyme": "str",
    "Prénoms": "str",
    "Date_Naissance": "str",
    "Ville_Naissance": "str",
    "Pays_Naissance": "str",
    "Nationalité": "str",
    "Adresse_Ligne1": "str",
    "Adresse_Ligne2": "str",
    "Adresse_Ligne3": "str",
    "Code_Postal": "str",
    "Ville": "str",
    "Code_Commune": "str",
    "Pays": "str",
    "Activité_Forain": "str",
    "EIRL": "str",
    "Auto-entrepreneur": "str",
    "DAP": "str",
    "DAP_Dénomination": "str",
    "DAP_Objet": "str",
    "DAP_Date_Clôture": "str",
    "DAP_Adresse_Ligne1": "str",
    "DAP_Adresse_Ligne2": "str",
    "DAP_Adresse_Ligne3": "str",
    "DAP_Code_Postal": "str",
    "DAP_Ville": "str",
    "DAP_Code_Commune": "str",
    "DAP_Pays": "str",
    "Conjoint_Collab_Nom_Patronym": "str",
    "Conjoint_Collab_Nom_Usage": "str",
    "Conjoint_Collab_Pseudo": "str",
    "Conjoint_Collab_Prénoms": "str",
    "Conjoint_Collab_Date_Fin": "str",
    "Date_Greffe": "str",
    "Libelle_Evt": "str",
}

parse_dates = [
    "Date_Immatriculation",
    "Date_1re_Immatriculation",
    "Date_Radiation",
    "Date_Transfert",
    "Date_Debut_Activité",
    "Date_Début_1re_Activité",
    "Date_Cessation_Activité",
    "Date_Naissance",
    "DAP_Date_Clôture",
    "Date_Greffe",
]

In [ ]:
option_extract = 'PP'
from sagemaker import get_execution_role
role = get_execution_role() 
instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
bucket = 'calfdata'
agg_datasock = inpi.inpiStock(instance_aws, bucket)
#https://calfdata.s3.eu-west-3.amazonaws.com/INPI/TC_1/Stock/Stock_initial/PP/0101_S1_20170504_3_PP.csv
agg_datasock.appendInpiStock(option_extract, dtype, parse_dates)

In [ ]:
filename = 'https://calfdata.s3.eu-west-3.amazonaws.com/INPI/TC_1/Stock/Stock_initial/PP/0101_S1_20170504_3_PP.csv'
pd.read_csv(filename, sep=";")

In [ ]:
inpi.appendData(
    origin="PP",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_stock,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="PP",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="PP",
    EVT=True,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

In [ ]:
df_pp_initial.loc[lambda x: x["Siren"].isin([827575812])].T

In [ ]:
df_pp_update.loc[lambda x: x["Siren"].isin([827575812])].T

## PM

In [ ]:
dtype = {
    "Code Greffe": "int",
    "Nom_Greffe": "str",
    "Numero_Gestion": "str",
    "Siren": "int",
    "Type_Inscription": "str",
    "Date_Immatriculation": "str",
    "Date_1re_Immatriculation": "str",
    "Date_Radiation": "str",
    "Date_Transfert": "str",
    "Sans_Activité": "str",
    "Date_Debut_Activité": "str",
    "Date_Début_1re_Activité": "str",
    "Date_Cessation_Activité": "str",
    "Denomination": "str",
    "Sigle": "str",
    "Forme_Juridique": "str",
    "Associé_Unique": "str",
    "Activité_Principale": "str",
    "Type_Capital": "str",
    "Capital": "float64",
    "Capital_Actuel": "float64",
    "Devise": "str",
    "Date_Clôture": "str",
    "Date_Clôture_Except.": "str",
    "Economie_Sociale_Solidaire": "str",
    "Durée_PM": pd.Int64Dtype(),
    "Date_Greffe": "str",
    "Libelle_Evt": "str",
}

parse_dates = [
    "Date_Immatriculation",
    "Date_1re_Immatriculation",
    "Date_Radiation",
    "Date_Transfert",
    "Date_Debut_Activité",
    "Date_Début_1re_Activité",
    "Date_Cessation_Activité",
    "Date_Clôture",
    "Date_Clôture_Except.",
    "Date_Greffe",
]

In [ ]:
inpi.appendData(
    origin="PM",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_stock,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="PM",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="PM",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

## REP

Doc REP: https://scm.saas.cagip.group.gca/PERNETTH/database/blob/master/Documentation/IMR/Doc_Tech_IMR_Mai_2019_v1.5.1.pdf
- page 30

Attention, certains champs sont remplis selon le type de la société. Ex SIREN -> Si type =PM

Pourquoi deux SIREN?

Le deuxième siren correspond a celui de la PM qui detient l'entreprise.

Ex: Peugeot a plusieurs détenteur. Le détenteur possède un siren, il sera affiché dans siren.1 uniquement si détenteur est une PM

In [ ]:
dtype = {
    "Code Greffe": "int",
    "Nom_Greffe": "str",
    "Numero_Gestion": "str",
    "Siren": "int",
    "Type": "str",
    "Nom_Patronymique": "str",
    "Nom_Usage": "str",
    "Pseudonyme": "str",
    "Prénoms": "str",
    "Dénomination": "str",
    "Siren.1": "str",
    "Forme_Juridique": "str",
    "Adresse_Ligne1": "str",
    "Adresse_Ligne2": "str",
    "Adresse_Ligne3": "str",
    "Code_Postal": "str",
    "Ville": "str",
    "Code_Commune": "str",
    "Pays": "str",
    "Date_Naissance": "str",
    "Ville_Naissance": "str",
    "Pays_Naissance": "str",
    "Nationalité": "str",
    "Qualité": "str",
    "Rep_Perm_Nom": "str",
    "Rep_Perm_Nom_Usage": "str",
    "Rep_Perm_Pseudo": "str",
    "Rep_Perm_Prénoms": "str",
    "Rep_Perm_Date_Naissance": "str",
    "Rep_Perm_Ville_Naissance": "str",
    "Rep_Perm_Pays_Naissance": "str",
    "Rep_Perm_Nationalité": "str",
    "Rep_Perm_Adr_Ligne1": "str",
    "Rep_Perm_Adr_Ligne2": "str",
    "Rep_Perm_Adr_Ligne3": "str",
    "Rep_Perm_Code_Postal": "str",
    "Rep_Perm_Ville": "str",
    "Rep_Perm_Code_Commune": "str",
    "Rep_Perm_Pays": "str",
    "Conjoint_Collab_Nom_Patronym": "str",
    "Conjoint_Collab_Nom_Usage": "str",
    "Conjoint_Collab_Pseudo": "str",
    "Conjoint_Collab_Prénoms": "str",
    "Conjoint_Collab_Date_Fin": "str",
    "ID_Représentant": "str",
    "date_greffe": "str",
    "Libelle_Evt": "str",
}

parse_dates = ["Date_Naissance", "date_greffe"]

In [ ]:
inpi.appendData(
    origin="REP",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_stock,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="REP",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="REP",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

## ETS

In [ ]:
dtype = {
    "Code Greffe": "int",
    "Nom_Greffe": "str",
    "Numero_Gestion": "str",
    "Siren": "int",
    "Type": "str",
    "Siège_PM": "str",
    "RCS_Registre": "str",
    "Adresse_Ligne1": "str",
    "Adresse_Ligne2": "str",
    "Adresse_Ligne3": "str",
    "Code_Postal": "str",
    "Ville": "str",
    "Code_Commune": "str",
    "Pays": "str",
    "Domiciliataire_Nom": "str",
    "Domiciliataire_Siren": "str",
    "Domiciliataire_Greffe": "str",
    "Domiciliataire_Complément": "str",
    "Siege_Domicile_Représentant": "str",
    "Nom_Commercial": "str",
    "Enseigne": "str",
    "Activité_Ambulante": "str",
    "Activité_Saisonnière": "str",
    "Activité_Non_Sédentaire": "str",
    "Date_Début_Activité": "str",
    "Activité": "str",
    "Origine_Fonds": "str",
    "Origine_Fonds_Info": "str",
    "Type_Exploitation": "str",
    "ID_Etablissement": "int",
    "Date_Greffe": "str",
    "Libelle_Evt": "str",
}

parse_dates = ["Date_Début_Activité", "Date_Greffe"]

In [ ]:
inpi.appendData(
    origin="ETS",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_stock,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="ETS",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="ETS",
    EVT=True,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

## ACTES

In [ ]:
dtype = {
    'Code Greffe':'int',
 'Nom_Greffe':'str',
 'Numero_Gestion':'str',
 'Siren':'int',
 'Date_Dépôt':'str',
 'Numéro_Dépôt':'str',
 'Date_Acte':'str',
 'Type_Acte':'str',
 'Décision':'str',
 'Etat':'str'
}

parse_dates = ["Date_Dépôt", "Date_Acte"]

In [ ]:
inpi.appendData(
    origin="ACTES",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_stock,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="ACTES",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="ACTES",
    EVT=True,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

## COMPTES ANNUELS

In [ ]:
dtype = {
    'Code Greffe':'int',
 'Nom_Greffe':'str',
 'Numero_Gestion':'str',
 'Siren':'int',
 'Date_Dépôt':'str',
 'Numéro_Dépôt':'str',
 'Date_Clôture':'str',
 'Durée_Exercice':pd.Int64Dtype(),
 'Type_Remise':'str',
 'Confidentialité':'str',
 'Confidentialité_Compte_résultat':'str',
 'Etat':'str'
}

parse_dates = ["Date_Dépôt", "Date_Clôture"]

In [ ]:
inpi.appendData(
    origin="COMPTES_ANNUELS",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_stock,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="COMPTES_ANNUELS",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="COMPTES_ANNUELS",
    EVT=True,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

## OBS

In [ ]:
dtype = {
    'Code Greffe':'int',
 'Nom_Greffe':'str',
 'Numero_Gestion':'str',
 'Siren':'int',
 'ID_Observation':'int',
 'Numéro_Observation':'int',
 'Date_Ajout':'str',
 'Date_Suppression':'str',
 'Texte':'str',
 'Date_Greffe':'str',
 'Etat ':'str'
}

parse_dates = ["Date_Ajout", "Date_Suppression", "Date_Greffe"]

In [ ]:
inpi.appendData(
    origin="OBS",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_stock,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="OBS",
    EVT=False,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)

In [ ]:
inpi.appendData(
    origin="OBS",
    EVT=True,
    dtype=dtype,
    parse_date=parse_dates,
    path=path_flux,
    list_unique=False,
)